In [1]:
#conda install in the correct environment
#!pip install eumdac

In [1]:
import eumdac
import datetime
import shutil
from datetime import timedelta
from sentinelsat import read_geojson, geojson_to_wkt
import zipfile
import gzip
import os
import pandas as pd
from glob import glob

download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

/mnt/beegfs/home/alexisa2019/.conda/envs/myCdom/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_datastore(consumer_key, consumer_secret):
    token = eumdac.AccessToken((consumer_key, consumer_secret))
    print(f"This token '{token}' expires {token.expiration}")
    return eumdac.DataStore(token)

def Sentinel_request(datastore, roi,
                     collectionID='EO:EUM:DAT:0409',
#                      geojson_file, 
                     opath = './',
                     start_date = None, end_date = None):

    WKT = 'POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi]))
    selected_collection = datastore.get_collection(collectionID)
    print(selected_collection.title)
    
    if(end_date == None):
        end_date = datetime.date.today()
    if(start_date == None):
        start_date = end_date-timedelta(days=1)
        
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    print('Searching from %s to %s' %(start_date, end_date))
    products = selected_collection.search(geo=WKT, 
                                          dtstart=start_date, 
                                          dtend=end_date)
    print(f'Found Datasets: {len(products)} datasets for the given time range')
    for product in products:
        with product.open() as fsrc, open(opath+fsrc.name, mode='wb') as fdst:
#             if os.path.isfile(fdst.name):
            if os.path.getsize(fdst.name)>0:

                print(f'Product {fdst.name} exists.')
            else:
                shutil.copyfileobj(fsrc, fdst)
                print(f'Download of product {product} finished.')
    print('All downloads are finished.')
            
    rpath = opath+'S3_%s_%s' % (start_date.strftime("%Y%m%d"),
                                start_date.strftime("%Y%m%d"))                    
    for f in glob(opath+'S3*.zip'):
        try:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                print('Unziping %s' %f)
                zip_ref.extractall(rpath)
                os.remove(f)
        except:
            print('Bad zip file: %s' %f)

    print(f'Done! Files in {rpath}')

In [ ]:
'''
def Sentinel_request(datastore,
                     roi, #instead of geojson file
                     #consumer_key, consumer_secret, # want to do auth and request separetely 
                                                     # so we can do several requests under same token
                     collectionID='EO:EUM:DAT:0409', # give a defaul value
                     #geojson_file, 
                     opath = './', #make key arg, shorter, default to local
                     start_date = None, end_date = None):

    #footprint = geojson_to_wkt(read_geojson(geojson_file))
    WKT = 'POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi]))
    
    selected_collection = datastore.get_collection(collectionID)
    print(selected_collection.title)
    
#     if(start_date == None or end_date == None):
#         start_date = yesterday.strftime("%Y%m%d")
#         end_date = today.strftime("%Y%m%d")
    
    if(end_date == None): #separate to be able to generate from start to today
        end_date = datetime.date.today()
    if(start_date == None):
        start_date = end_date-timedelta(days=1)
        
    start_date = pd.to_datetime(start_date) # make datetime object if input date is string
    end_date = pd.to_datetime(end_date) # make datetime object if input date is string
    
    print('Searching from %s to %s' %(start_date, end_date))
    products = selected_collection.search(geo=WKT, 
                                          dtstart=start_date, 
                                          dtend=end_date)
        
    print(f'Found Datasets: {len(products)} datasets for the given time range')
    for product in products:
        with product.open() as fsrc, open(opath+fsrc.name, mode='wb') as fdst: #include opath
            print(f'Opening {fdst} for download')
#             if os.path.isfile(fdst.name):
#                 print(f'Product {fdst.name} exists.')
#             else:
#                 shutil.copyfileobj(fsrc, fdst)
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {product} finished.')
    print('All downloads are finished.')
    #create folder name with request time range:
    rpath = opath+'S3_%s_%s' % (start_date.strftime("%Y%m%d"),
                                start_date.strftime("%Y%m%d"))
    
#     os.makedirs(rpath, exist_ok=True)
#     rpath = '/mnt/beegfs/home/alexisa2019/Projects/SentinelProject/products'
    
    #unzip and remove
    for f in glob(opath+'S3*.zip'):
        try:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                print('Unziping %s' %f)
                zip_ref.extractall(rpath)
        except:
            print('Bad zip file: %s' %f)
        os.remove(f)
    print(f'Done! Files in {rpath}')'''

In [9]:
cd Sentinel 

/mnt/beegfs/home/alexisa2019/Projects/SentinelProject


In [10]:
ls

GOMEX_FB.geojson  Sentinel DL 2.ipynb  flbay.geojson
LakeO.geojson     Sentinel DL.ipynb    gomex.geojson
README.md         Sentinel Vis.ipynb   products/


In [12]:
consumer_key = 'yawsJIqGoAetBf_MqpcByUwnrWMa'
consumer_secret = 'w6nWuFAtEgPhFESfK7Q4MzOs9t0a'

roi = [[-81.0955810546875,26.675685969067487],
       [-80.54489135742188,26.675685969067487],
       [-80.54489135742188,27.243641579169292],
       [-81.0955810546875,27.243641579169292],
       [-81.0955810546875,26.675685969067487]]

datastore = get_datastore(consumer_key, consumer_secret)
Sentinel_request(datastore, roi, opath='./products/', start_date=datetime.date(2022,3,23),
                 end_date=datetime.date(2022,3,24))

This token '2e532d2e-0033-33ec-8dbb-2ba6f29c8ef9' expires 2022-08-25 01:48:27.579249
OLCI Level 1B Full Resolution - Sentinel-3
Searching from 2022-03-23 00:00:00 to 2022-03-24 00:00:00
Found Datasets: 1 datasets for the given time range
Download of product S3B_OL_1_EFR____20220323T152738_20220323T153038_20220324T205601_0179_064_068_2520_MAR_O_NT_002.SEN3 finished.
All downloads are finished.
Unziping ./products/S3B_OL_1_EFR____20220323T152738_20220323T153038_20220324T205601_0179_064_068_2520_MAR_O_NT_002.SEN3.zip
Done! Files in ./products/S3_20220323_20220323


In [2]:
# consumer_key = 'yawsJIqGoAetBf_MqpcByUwnrWMa'
# consumer_secret = 'w6nWuFAtEgPhFESfK7Q4MzOs9t0a'

# credentials = (consumer_key, consumer_secret)

# token = eumdac.AccessToken(credentials)

In [3]:
# create data store object
# datastore = eumdac.DataStore(token)

# show all collections:
for collectionID in datastore.collections:
    if 'Sentinel-3' in collectionID.title:
        print(f"{collectionID}: {collectionID.title}")

EO:EUM:DAT:0556: OLCI Level 2 Ocean Colour Full Resolution (version BC003) - Sentinel-3 - Reprocessed
EO:EUM:DAT:0411: SLSTR Level 1B Radiances and Brightness Temperatures - Sentinel-3
EO:EUM:DAT:0407: OLCI Level 2 Ocean Colour Full Resolution - Sentinel-3
EO:EUM:DAT:0409: OLCI Level 1B Full Resolution - Sentinel-3
EO:EUM:DAT:0557: OLCI Level 2 Ocean Colour Reduced Resolution (version BC003) - Sentinel-3 - Reprocessed
EO:EUM:DAT:0412: SLSTR Level 2 Sea Surface Temperature (SST) - Sentinel-3
EO:EUM:DAT:0416: SLSTR Level 2 Aerosol Optical Depth - Sentinel-3
EO:EUM:DAT:0406: SRAL Level 1B - Sentinel-3
EO:EUM:DAT:0415: SRAL Level 2 Altimetry Global - Sentinel-3
EO:EUM:DAT:0413: SRAL Level 1A Unpacked L0 Complex echos - Sentinel-3
EO:EUM:DAT:0414: SRAL Level 1B stack echoes - Sentinel-3
EO:EUM:DAT:0410: OLCI Level 1B Reduced Resolution - Sentinel-3
EO:EUM:DAT:0408: OLCI Level 2 Ocean Colour Reduced Resolution - Sentinel-3


In [4]:
# collectionID = 'EO:EUM:DAT:0407' # set collection ID for OLCI L2 WFR
collectionID = 'EO:EUM:DAT:0409' # set collection ID for OLCI L1

# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

OLCI Level 1B Full Resolution - Sentinel-3


In [5]:
roi = [[-81.0955810546875,26.675685969067487],
       [-80.54489135742188,26.675685969067487],
       [-80.54489135742188,27.243641579169292],
       [-81.0955810546875,27.243641579169292],
       [-81.0955810546875,26.675685969067487]]

WKT = 'POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi]))

In [6]:
# time filter the collection for products
prodList = []
start = datetime.date(2022, 3, 23)
end = datetime.date(2022, 3, 24)
products = selected_collection.search(geo=WKT, dtstart=start, dtend=end)

for product in products:
    print(product)
    prodList.append(product)

S3A_OL_1_EFR____20220324T154054_20220324T154354_20220325T194954_0179_083_225_2520_MAR_O_NT_002.SEN3
S3B_OL_1_EFR____20220324T150127_20220324T150427_20220325T201204_0180_064_082_2520_MAR_O_NT_002.SEN3
S3B_OL_1_EFR____20220323T152738_20220323T153038_20220324T205601_0179_064_068_2520_MAR_O_NT_002.SEN3


In [22]:
product = prodList[0]

selected_product = datastore.get_product(product_id=str(product), collection_id=collectionID)

with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
    print(f'Downloading {fsrc.name}')
    shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {fsrc.name} finished.')
                
#         os.remove(filePath)

Download of product S3A_OL_1_EFR____20220324T154054_20220324T154354_20220325T194954_0179_083_225_2520_MAR_O_NT_002.SEN3.zip finished.


BadZipFile: File is not a zip file

In [23]:
filePath = os.path.join(os.getcwd(), "products/S3A_OL_1_EFR____20220324T154054_20220324T154354_20220325T194954_0179_083_225_2520_MAR_O_NT_002.SEN3.zip")
   
dl_dir = os.path.join(os.getcwd(), "products")
                            
with zipfile.ZipFile(filePath, 'r') as zip_ref:
    zip_ref.extractall(dl_dir)